# Exercise: Geographical Cluster Analysis of Taxi Rides
Using the NY Taxi data set (see Use Case Block I) and the use case from the lecture...

In [11]:
import pandas as pd
import numpy as np
import folium

In [12]:
# we load the data we have saved after wrangling and pre-processing in block I
train=pd.read_csv('../../DATA/train_cleaned.csv')

In [13]:
#select only the culumns with the ride coordinates
coordinates = train[ ['pickup_latitude','pickup_longitude','dropoff_latitude' , 'dropoff_longitude' ] ]

## Clustering approach from the lecture
we will be using simple K-Means:
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [55]:
from sklearn.cluster import KMeans

In [56]:
#define number of clusters and create instance
clusters=50
myKMeans=KMeans(n_clusters=clusters, n_jobs=-1)#parallelize to all cores

In [57]:
%%time
#train model
myKMeans.fit(coordinates.to_numpy()[:100000,:])#use only subset of the data to make it faster

C:\Users\togie\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:938: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


Wall time: 35.4 s


KMeans(n_clusters=50, n_jobs=-1)

In [59]:
#get cluster centers
centers=myKMeans.cluster_centers_
    

In [60]:
#draw map: green: start, red: end
cluster_map = folium.Map(location = [40.730610,-73.935242],zoom_start = 12,)
for i in range(clusters):
    folium.CircleMarker([centers[i,0], centers[i,1]], radius=3,                
                        color="green", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
    folium.CircleMarker([centers[i,2], centers[i,3]], radius=3,                
                        color="red", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
    folium.PolyLine([ [centers[i,0],centers[i,1]] , [centers[i,2],centers[i,3]]  ], color="black", weight=2.5, opacity=1).add_to(cluster_map)

In [61]:
cluster_map

## Exercise 1
Write a function ```show_cluster(cluster_number,...)``` that draws the cluster centers and all start and end points of a given cluster in the map.
* use the ```predict()``` method to map all data in ```train_data``` to a cluster center
* use ```folium.CircleMarker``` to draw all members of a given cluster


In [138]:
def show_cluster(cluster_number,model,train_data):
    thiscluster_map = folium.Map(location = [40.730610,-73.935242],zoom_start = 12,)
    
    #Datensatz verkleinern
    train_data=train_data.to_numpy()[:100000,:]
    
    cluster_label=model.labels_[cluster_number]
    centers=model.cluster_centers_
    
    #predict
    pred = model.predict(train_data)
    
    #Draw all members of the given cluster
    for i in range(pred.shape[0]):
        if pred[i] == cluster_number:
            folium.CircleMarker([train_data[i,0], train_data[i,1]], radius=1,                
                                color="blue", 
                                fill_opacity=0.9
                               ).add_to(thiscluster_map)
            folium.CircleMarker([train_data[i,2], train_data[i,3]], radius=1,                
                                color="blue", 
                                fill_opacity=0.9
                               ).add_to(thiscluster_map)
            folium.PolyLine([ [train_data[i,0],train_data[i,1]] , [train_data[i,2],train_data[i,3]]  ], color="blue", weight=1.5, opacity=0.5).add_to(thiscluster_map)
    
    #Draw Center
    folium.CircleMarker([centers[cluster_number,0], centers[cluster_number,1]], radius=3,                
                        color="green", 
                        fill_opacity=0.9
                       ).add_to(thiscluster_map)
    folium.CircleMarker([centers[cluster_number,2], centers[cluster_number,3]], radius=3,                
                        color="red", 
                        fill_opacity=0.9
                       ).add_to(thiscluster_map)
    folium.PolyLine([ [centers[cluster_number,0],centers[cluster_number,1]] , [centers[cluster_number,2],centers[cluster_number,3]]  ], color="black", weight=2.5, opacity=1).add_to(thiscluster_map)
    
    return thiscluster_map

In [140]:
show_cluster(28,myKMeans,coordinates)

## Exercise 2
Write a function ```cluster_var(cluster_number,...)``` that computes the intra- and extra cluster variance for a given cluster. Apply it to all clusters and compare the results for k=100 and k=10.